In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# Modeling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor 
import warnings
# warnings.filterwarnings('ignore')

In [30]:
import sklearn
print(sklearn.__version__)

1.7.1


In [31]:
file = "data/eq_maintenance_updated_data.csv"
df = pd.read_csv(file)
df.head(1)

,equipment_id,equipment_type,location,install_date,last_service_date,next_scheduled_service,age_days,runtime_hours,temperature,vibration_level,power_consumption_kw,humidity_level,error_codes_count,manual_override,downtime_last_30d,service_priority,failure_within_7_days
0,dda080ad-7ead-4676-92e2-b6d818addfcc,HVAC,Building A,2023-08-04,2024-01-26,2024-03-31,742,5496.71,73.62,3.97,27.62,39.36,0,0,0.12,Medium,0


#### Prpairing X and Y

In [32]:
X = df.drop(columns=('failure_within_7_days'), axis=1)

In [33]:
y = df['failure_within_7_days']

In [34]:
# print("Categories in 'equipment_id' variable:     ",end=" " )
# print(df['equipment_id'].unique())

print("Categories in 'equipment_type' variable:  ",end=" ")
print(df['equipment_type'].unique())

print("Categories in'location' variable:",end=" " )
print(df['location'].unique())

print("Categories in 'install_date' variable:     ",end=" " )
print(df['install_date'].unique())

print("Categories in 'last_service_date' variable:     ",end=" " )
print(df['last_service_date'].unique())

print("Categories in 'next_scheduled_service' variable:     ",end=" " )
print(df['next_scheduled_service'].unique())

print("Categories in 'service_priority' variable:     ",end=" " )
print(df['service_priority'].unique())

Categories in 'equipment_type' variable:   ['HVAC' 'Boiler' 'Elevator' 'Generator' 'Pump']
Categories in'location' variable: ['Building A' 'Building C' 'Building B' 'Warehouse 2' 'Warehouse 1']
Categories in 'install_date' variable:      ['2023-08-04' '2022-12-26' '2022-11-14' '2023-11-11' '2024-06-14'
 '2021-09-06' '2022-10-31' '2023-09-19' '2021-04-11' '2023-07-17'
 '2023-05-13' '2021-09-11' '2024-02-29' '2021-09-04' '2022-08-07'
 '2020-11-17' '2023-05-08' '2022-06-10' '2023-12-28' '2024-06-08'
 '2024-02-20' '2020-12-19' '2021-04-14' '2022-03-22' '2021-08-13'
 '2023-06-16' '2021-07-30' '2022-07-07' '2022-04-27' '2021-08-14'
 '2021-08-16' '2024-07-01' '2024-01-31' '2024-04-30' '2021-03-30'
 '2022-12-17' '2024-01-11' '2022-12-09' '2021-11-21' '2022-06-15'
 '2020-12-01' '2022-02-07' '2023-04-18' '2021-03-31' '2022-07-30'
 '2020-08-21' '2022-11-13' '2021-03-20' '2021-06-19' '2023-08-15'
 '2020-09-21' '2021-04-06' '2023-11-21' '2021-12-08' '2021-12-13'
 '2021-01-21' '2022-03-02' '2024-04-

In [35]:
# Create columns transformer with 3 types of transformers

num_feautres = X.select_dtypes(exclude=['object']).columns
cat_features = X.select_dtypes(include=['object']).columns  

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer   
from sklearn.pipeline import Pipeline

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        
        ('OneHotEncoder', oh_transformer, cat_features),
        ('StandardScaler', numeric_transformer, num_feautres)
    ],
    remainder='passthrough'
)

In [36]:
X = preprocessor.fit_transform(X)

In [37]:
# Separate the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 3130), (200, 3130), (800,), (200,))

#### Create an evaluation function to give all matrix after model training

In [40]:
def evaluate_model(true, predicted) :
    mse = mean_squared_error(true, predicted)
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)
    
    return mae, rmse, r2

In [44]:
models = {
    "LinearRegression": LinearRegression(),
    "KNeighborsRegressor": KNeighborsRegressor(),   
    "Lasso": sklearn.linear_model.Lasso(),
    "Ridge": sklearn.linear_model.Ridge(),
    "DecissionTree": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),   
    "XGBRegressor": XGBRegressor(),
    "CatBoostRegressor": CatBoostRegressor(verbose=0),
    "adaboost": sklearn.ensemble.AdaBoostRegressor(),
    # "SVR": SVR()  # Support Vector Regressor
    }
model_list = []
r2_list = []
for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train the model

    # Make a prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluate Train and Test dataset
    # model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print("--------------------------------------------------")
    print(list(models.keys())[i]) 
    model_list.append(list(models.keys())[i])

    print('Model Performance on TRaining Data')
    print("_ Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("_ Mean Absolute Error: {:.4f}".format(model_train_mae))  
    print("_ R2 Score: {:.4f}".format(model_train_r2))

    print("--------------------------------------------------")
    print("Model performance on Test Data")
    print("_ Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("_ Mean Absolute Error: {:.4f}".format(model_test_mae)) 
    print("_ R2 Score: {:.4f}".format(model_test_r2)) 

    r2_list.append(model_test_r2)
    print('=' *35)
    print('\n')


--------------------------------------------------
LinearRegression
Model Performance on TRaining Data
_ Root Mean Squared Error: 0.0000
_ Mean Absolute Error: 0.0000
_ R2 Score: 1.0000
--------------------------------------------------
Model performance on Test Data
_ Root Mean Squared Error: 0.2413
_ Mean Absolute Error: 0.1306
_ R2 Score: 0.0417


--------------------------------------------------
KNeighborsRegressor
Model Performance on TRaining Data
_ Root Mean Squared Error: 0.1586
_ Mean Absolute Error: 0.0453
_ R2 Score: 0.3988
--------------------------------------------------
Model performance on Test Data
_ Root Mean Squared Error: 0.2168
_ Mean Absolute Error: 0.0630
_ R2 Score: 0.2267


--------------------------------------------------
Lasso
Model Performance on TRaining Data
_ Root Mean Squared Error: 0.2045
_ Mean Absolute Error: 0.0837
_ R2 Score: 0.0000
--------------------------------------------------
Model performance on Test Data
_ Root Mean Squared Error: 0.2474


In [45]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2 Score']).sort_values(by='R2 Score', ascending=False)

,Model Name,R2 Score
7,CatBoostRegressor,0.917089
8,adaboost,0.843647
6,XGBRegressor,0.783061
5,RandomForestRegressor,0.748581
4,DecissionTree,0.670917
1,KNeighborsRegressor,0.226656
3,Ridge,0.091581
0,LinearRegression,0.041714
2,Lasso,-0.007430
